In [11]:
from ape import networks, Contract
import pandas as pd

from helpers.coingecko import get_votium_prices
from helpers.votium import fxs_emissions_usd
from helpers.subgraph_endpoints import *
from helpers.formatter import pct_format, dollar_format

In [ ]:
networks.parse_network_choice("ethereum:mainnet").__enter__()

In [13]:
cvx_price, crv_price, badger_price, fxs_price = get_votium_prices()
data = [
    [
        "${:.2f}".format(badger_price),
        "${:.2f}".format(crv_price),
        "${:.2f}".format(cvx_price),
        "${:.2f}".format(fxs_price),
    ]
]
headers = ["BADGER price", "CRV price", "CVX price", "FXS price"]
df_prices = pd.DataFrame(data, columns=headers)
df_prices


,BADGER price,CRV price,CVX price,FXS price
0,$2.34,$0.55,$3.18,$5.01


In [14]:
# bribes allocation
bribe_for_badgerfraxbp = 6_000
bribe_usd_val = bribe_for_badgerfraxbp * badger_price
data = [[bribe_for_badgerfraxbp, bribe_usd_val]]
headers = ["Bribe allocated to badgerfraxbp pool (BADGER)", "Bribe Value (USD)"]
df_bribes_allocation = pd.DataFrame(data, columns=headers)
df_bribes_allocation

,Bribe allocated to badgerfraxbp pool (BADGER),Bribe Value (USD)
0,6000,14040.0


In [15]:
# treasury capture
tvl_badgerfraxbp = get_tvl_curve_badgerfraxbp_pool()
badgerfraxbp_pool = Contract("0x09b2E090531228d1b8E3d948C73b990Cb6e60720")
pool_supply = badgerfraxbp_pool.totalSupply() / 1e18
private_vault_treasury = "0xa895B89D74a6BC23a284a0526e123ea776674cF5"
gauge_frax = Contract("0x5a92EF27f4baA7C766aee6d751f754EBdEBd9fae")
treasury_tcl_bal = gauge_frax.lockedLiquidityOf(private_vault_treasury) / 1e18
vault_owned_tcl = tvl_badgerfraxbp * (treasury_tcl_bal / pool_supply)
pool_capture = vault_owned_tcl / tvl_badgerfraxbp
treasury_fxs_gauge_capture = treasury_tcl_bal / (
    gauge_frax.totalLiquidityLocked() / 1e18
)
data = [[pct_format(pool_capture), pct_format(treasury_fxs_gauge_capture)]]
headers = ["BADGERFRAXBP Pool Capture", "Treasury Gauge Capture"]
df_capture = pd.DataFrame(data, columns=headers)
df_capture

,BADGERFRAXBP Pool Capture,Treasury Gauge Capture
0,31.779%,31.786%


In [16]:
# fxs weight voting to gauge at latest block
frax_gauge_controller = Contract("0x3669C421b77340B2979d1A00a792CC2ee0FcE737")
fxs_relative_weight = frax_gauge_controller.gauge_relative_weight(gauge_frax) / 1e18
gauge_earnings = fxs_emissions_usd(
    fxs_price, fxs_relative_weight, treasury_fxs_gauge_capture, biweekly=False
)
treasury_roi = gauge_earnings / bribe_usd_val
data = [[pct_format(fxs_relative_weight), dollar_format(gauge_earnings), pct_format(treasury_roi)]]
headers = ["Gauge Relative Weight", "Treasury yield earned (USD) - weekly", "Treasury ROI - biweekly"]
df_earnings_badger_pool = pd.DataFrame(data, columns=headers)
df_earnings_badger_pool

,Gauge Relative Weight,Treasury yield earned (USD) - weekly,Treasury ROI - biweekly
0,5.488%,$3173.3751,22.602%


In [23]:
# potential extra weight brought by the bribes (assuming full support of fxs whales and alike)
vefxs = Contract("0xc8418aF6358FFddA74e09Ca9CC3Fe03Ca6aDC5b0")
vefxs_ts = vefxs.totalSupply() / 1e18
vefxs_fxs_whale_bal = (
    vefxs.balanceOf("0x88e863d4572d2dae27db81e98837a9dbeb0e7a12") / 1e18
)
vefxs_fxs_whale_controlled = vefxs_fxs_whale_bal / vefxs_ts
fxs_relative_weight_after_bribe =  vefxs_fxs_whale_controlled + fxs_relative_weight
gauge_earnings = fxs_emissions_usd(
    fxs_price, fxs_relative_weight_after_bribe, treasury_fxs_gauge_capture, biweekly=False
)
treasury_roi = gauge_earnings / bribe_usd_val
data = [[pct_format(fxs_relative_weight_after_bribe), dollar_format(gauge_earnings), pct_format(treasury_roi)]]
headers = ["Gauge Relative Weight", "Treasury yield earned (USD) - weekly", "Treasury ROI - biweekly"]
df_earnings_badger_pool = pd.DataFrame(data, columns=headers)
df_earnings_badger_pool

,Gauge Relative Weight,Treasury yield earned (USD) - weekly,Treasury ROI - biweekly
0,11.605%,$6710.7005,47.797%
